In [1]:
import pandas as pd
import numpy as np
#import seaborn as sns
import math
from neuralprophet import NeuralProphet

In [2]:
file = 'df4eda.csv'
df4eda = pd.read_csv(file, index_col=0)
df4eda

WARNING - (py.warnings._showwarnmsg) - /tmp/ipykernel_124/2940781367.py:2: DtypeWarning: Columns (22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df4eda = pd.read_csv(file, index_col=0)



,ProductID,Date,Zip,Units,Revenue_T,Manufacturer,Category,Segment,Product,isVanArsdel,...,Running Months,Rolling Period,Rolling Period Sort,MonthIndex,City,State,Region,District,ManufacturerID_y,MfgisVanArsdel
0,981,2014-12-30,6111,12,31676.40,Natura,Urban,Convenience,Natura UC-44,No,...,Running 12 Months,P-00,12.0,192,"Newington, CT",CT,East,District #02,8,No
1,1086,2014-12-30,92879,7,11020.59,Pirum,Rural,Productivity,Pirum RP-32,No,...,Running 12 Months,P-00,12.0,192,"Corona, CA",CA,West,District #38,10,No
2,1085,2014-12-30,92879,7,11020.59,Pirum,Rural,Productivity,Pirum RP-31,No,...,Running 12 Months,P-00,12.0,192,"Corona, CA",CA,West,District #38,10,No
3,489,2014-12-30,32901,3,35010.36,VanArsdel,Urban,Moderation,Maximus UM-94,Yes,...,Running 12 Months,P-00,12.0,192,"Melbourne, FL",FL,East,District #10,7,Yes
4,1989,2014-12-30,84770,3,16441.11,Currus,Rural,Select,Currus RS-08,No,...,Running 12 Months,P-00,12.0,192,"Saint George, UT",UT,West,District #38,4,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,990,2002-04-30,11752,1,5291.37,Natura,Urban,Convenience,Natura UC-53,No,...,NaN,NaN,NaN,40,"Islip Terrace, NY",NY,East,District #03,8,No
1048571,990,2002-04-30,39367,1,5291.37,Natura,Urban,Convenience,Natura UC-53,No,...,NaN,NaN,NaN,40,"Waynesboro, MS",MS,Central,District #24,8,No
1048572,2169,2002-04-30,98311,1,6614.37,Victoria,Urban,Extreme,Victoria UE-22,No,...,NaN,NaN,NaN,40,"Bremerton, WA",WA,West,District #34,14,No
1048573,1144,2002-04-30,95018,1,4976.37,Pirum,Urban,Regular,Pirum UR-01,No,...,NaN,NaN,NaN,40,"Felton, CA",CA,West,District #36,10,No


In [3]:
df = df4eda.copy()
#df.drop(['Zip', 'Region', 'Quarter', 'MonthName', 'Segment', 'RunningMonths', 'isVanArsdel',
#                   'MfgisVanArsdel', 'District', 'MonthIndex', 'MonthID', 'Month',
#                    'IsCompeteHide', 'ManufacturerID_y', 'Rolling Period Sort', 'Running Year',
#                   'Manufacturer', 'Product', 'Rolling Period', 'Running Months', 'ProductID',
#        'Units', 'Category', 'ManufacturerID_x', 'IsCompete', 'MonthNo', 'Year', 'City', 'State'], axis=1, inplace=True)
df = df[['Date','Revenue_T']]
df = df.rename(columns={"Date": "ds"})

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1048575 entries, 0 to 1048574
Data columns (total 2 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   ds         1048575 non-null  object 
 1   Revenue_T  1048534 non-null  float64
dtypes: float64(1), object(1)
memory usage: 24.0+ MB


In [5]:
# удаление дубликатов
df = df.drop_duplicates()

#агрегация и суммирование по датам
agg_func_math = {
    'Revenue_T': ['sum']
}

# группирую по дате (месяц-год) и считаю статистические величины и все округляю до второго знака
df = df.groupby(['ds']).agg(agg_func_math).round(2).reset_index()

In [ ]:
duplicate_rows_df = df[df.duplicated()]
print(duplicate_rows_df)
print("number of duplicate rows: ", duplicate_rows_df.shape)

In [6]:
# в новом датасете создам из названий мультииндексных родительских, объединяя через нижнее подчеркивание
df.columns = ['_'.join(col) for col in df.columns.values]

# сброшу индексы
df = df.reset_index()
df

,index,ds_,Revenue_T_sum
0,0,2002-04-30,954560.88
1,1,2002-05-01,501617.97
2,2,2002-05-02,460054.35
3,3,2002-05-03,212396.94
4,4,2002-05-05,634911.48
...,...,...,...
4475,4475,2014-12-26,894406.59
4476,4476,2014-12-27,1072392.93
4477,4477,2014-12-28,1174332.60
4478,4478,2014-12-29,420040.53


In [7]:
df = df.rename(columns={"ds_": "ds", "Revenue_T_sum": "y"})
df.drop(['index'], axis=1, inplace=True)
df

,ds,y
0,2002-04-30,954560.88
1,2002-05-01,501617.97
2,2002-05-02,460054.35
3,2002-05-03,212396.94
4,2002-05-05,634911.48
...,...,...
4475,2014-12-26,894406.59
4476,2014-12-27,1072392.93
4477,2014-12-28,1174332.60
4478,2014-12-29,420040.53


In [ ]:
#df['ds'] = df4eda['Date']
#df['x'] = df4eda['Revenue_T']
#df = df['ds'].concat(df['x'])
#df

In [8]:
m = NeuralProphet()

metrics = m.fit(df, freq="D")

forecast = m.predict(df)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 97.21% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 32
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 126
WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (140) is too small than the required number for the learning rate finder (241). The results might not be optimal.
Missing logger folder: /home/jupyter/Diplom/lightning_logs


Finding best initial lr:   0%|          | 0/241 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 97.21% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 97.21% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D


Predicting: 140it [00:00, ?it/s]

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


In [9]:
from prophet import Prophet

m = Prophet()
m.fit(df)

future = m.make_future_dataframe(periods=365)
future.tail()

17:41:23 - cmdstanpy - INFO - Chain [1] start processing
17:41:23 - cmdstanpy - INFO - Chain [1] done processing


,ds
4840,2015-12-26
4841,2015-12-27
4842,2015-12-28
4843,2015-12-29
4844,2015-12-30


In [ ]:
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)